In [1]:
import kratos
from agno.agent import Agent
from agno.models.lmstudio import LMStudio
from agno.tools.youtube import YouTubeTools
from agno.tools.yfinance import YFinanceTools

# Creating a few agents using the Agno framework for building agents
# All the llm models here use LMStudio as the provider and connect to local API server
# I'm using qwen/qwen3-4b model for both agents, but you can use any other model that you have access to.

finance_agent = Agent(
    model=LMStudio(
        id="qwen/qwen3-4b",
        base_url="http://host.docker.internal:1234/v1",
    ),
    name="KratosFinance",
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True)],
    instructions="You are Kratos Finance Agent, specialized in stock market analysis and financial data. Use YFinance to get current stock prices, analyst recommendations, and company information. When users ask about stocks, always provide current prices and relevant financial metrics. Be precise with numbers and explain what the data means for investment decisions.",
)


youtube_agent = Agent(
    model=LMStudio(
        id="qwen/qwen3-4b",
        base_url="http://host.docker.internal:1234/v1",
    ),
    name="KratosYouTube",
    tools=[YouTubeTools()],
    instructions="You are Kratos YouTube Agent, specialized in analyzing YouTube videos and transcripts. Use YouTube tools to extract video information, transcripts, and analyze content. When users mention YouTube videos, provide detailed analysis of the content, key points, and insights from the transcript. Be thorough in your analysis and highlight important information.",
)


In [2]:
# Submit these agents so that they are ready to be used at a later point - none of
# these models are actually loaded in memory right now and thus no resources are wasted.

print(kratos.submit(finance_agent, name="agent-finance", dependencies=["yfinance"]))
print(kratos.submit(youtube_agent, name="agent-youtube", dependencies=["youtube-transcript-api"]))

agent-finance
agent-youtube


In [3]:
# The name parameter we used in submit() acts as an identifier as to which agent do you want to invoke.

kratos.invoke(name="agent-finance", task="How has the trend been for NVIDIA stock?")

<think>
Okay, the user is asking about the trend for NVIDIA's stock. Let me figure out how to approach this.
First, I need to determine what information is required. The user wants to know the trend, which likely involves recent performance, price movements, and maybe analyst opinions. 
I should start by getting the current stock price for NVIDIA. The function get_current_stock_price is necessary here. The symbol for NVIDIA is NVDA, so I'll use that.
Next, to understand the trend, I might need historical data, but the available functions don't include historical prices. However, the user might be interested in the latest analyst recommendations, so using get_analyst_recommendations could provide insights into expert opinions. Also, company information from get_company_info might offer context, like recent developments affecting the stock.
Wait, but the user specifically asked for the trend. Since the tools don't have a function for historical data, maybe I can only provide the current 

In [4]:
kratos.invoke(name="agent-youtube", task="Summarize the transcript of this video: https://www.youtube.com/watch?v=dQw4w9WgXcQ")

<think>
Okay, the user wants me to summarize the transcript of the YouTube video with the URL https://www.youtube.com/watch?v=dQw4w9WgXcQ. Let me think about how to approach this.
First, I need to get the transcript of the video. The available tools include getting the captions, video data, and timestamps. Since the user asked for a transcript summary, I should start by retrieving the captions using the get_youtube_video_captions function. That function requires the video URL, which is provided.
Once I have the captions, I can process them to create a summary. But wait, maybe I should also check the video data to get more context? The get_youtube_video_data function gives details like title and author, which might be helpful for the summary. However, the user specifically mentioned summarizing the transcript, so the main focus should be on the captions.
So the steps are: first call get_youtube_video_captions with the given URL to fetch the transcript text. Then, analyze the captions to

In [5]:
from concurrent.futures import ThreadPoolExecutor, wait

print(f"\n🚀 Starting 3 concurrent invocations where each instance of an agent will have it's own model loaded in memory")

tasks = [
    "Stock price of NVIDIA",
    "Stock price of Apple",
    "Stock price of Tesla",
]

with ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(kratos.invoke, 'agent-finance', task, False) for task in tasks]

    print(f"\n🔄 Waiting for all agents to complete...")
    wait(futures)

    print(f"\n🔄 All agents have completed their tasks")

print("\n\nQ&A:")


🚀 Starting 3 concurrent invocations where each instance of an agent will have it's own model loaded in memory

🔄 Waiting for all agents to complete...

🔄 All agents have completed their tasks


Q&A:
